<font size=5> Social distancing: evidence of privilege in a pandemic from smartphones </font><br>
Nabarun Dasgupta, MPH, PhD | nab@unc.edu | @nabarund<br>
*Dr. Dasgupta is an epidemiologist at the University of North Carolina in Chapel Hill. He studies population level patterns of infectious disease, medication safety, and opioids.*<br>
Thanks to Ben White for data munging help. Code available on [GitHub](https://github.com/opioiddatalab/covid).<br>

**Co-authors**<br>
Michele Jonsson Funk, PhD<br>
Allison Lazard, PhD<br>
Benjamin Eugene White<br>
Steve W. Marshall, PhD<br>

<div class="alert alert-warning">

**Warning:** Code and output provided for evaluation only. Paper under review. Results have not been peer-reviewed and should not be used for clinical or public health interpretation.

</div>

On March 23, 2020 Stuart Thompson and Yaryna Serkez of *The New York Times* [published](https://www.nytimes.com/interactive/2020/03/23/opinion/coronavirus-economy-recession.html) a fascinating use of cell phone GPS signal information to gauge movement and commuting, during the advent of social distancing. They compared the state-level data in a [slick graphic](https://www.nytimes.com/interactive/2020/03/23/opinion/coronavirus-economy-recession.html) to political leanings. But we wanted to understand more about other community level characteristics of slow versus fast adopters.<br>
<br>
We were provided access to the same location dataset on social distancing published today in the. We used a data merging approach we have [previously published](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6339867/). Repurposing code from an ongoing project, we merged in community-level data from the Robert Wood Johnson Foundation's [County Health Rankings](https://www.countyhealthrankings.org/). This very rich dataset contains dozens of explanatory variables about health, social, and economic indicators.<br>

---

In [22]:
display "Notebook generated on $S_DATE at $S_TIME ET"

Notebook generated on 22 Apr 2020 at 08:51:49 ET


# Models

In [23]:
// Load pre-procesed data
clear all
set scheme economist
use "https://github.com/opioiddatalab/covid/blob/master/analysiset.dta?raw=true"

// Create results frame
frame create results str20 strat level avg LL UL

// Program to invert differences
program define invert, rclass
    version 16
    args ee ll ul
    
    di round((1-(1/`ee'))*-100,.1)
    di "LL: " round((1-(1/`ll'))*-100,.1)
    di "UL: " round((1-(1/`ul'))*-100,.1)
end

// Set up negative binomial model
program define modelrun, rclass
    version 16
    syntax varlist(numeric)
    
    frame change default
    foreach var of local varlist {
    
    di "----- RURALITY-ADJUSTED NEGBIN MODEL -----"
       nbreg `var' levels* homeorder i.rucc, irr nocons vce(r)

    * Store results
        frame post results ("`var'") (1) (round((r(table)[1,1]),.1)) (round((r(table)[5,1]),.1)) (round((r(table)[6,1]),.1))
        frame post results ("`var'") (2) (round((r(table)[1,2]),.1)) (round((r(table)[5,2]),.1)) (round((r(table)[6,2]),.1))
        frame post results ("`var'") (3) (round((r(table)[1,3]),.1)) (round((r(table)[5,3]),.1)) (round((r(table)[6,3]),.1))
        frame post results ("`var'") (4) (round((r(table)[1,4]),.1)) (round((r(table)[5,4]),.1)) (round((r(table)[6,4]),.1))
        frame post results ("`var'") (5) (round((r(table)[1,5]),.1)) (round((r(table)[5,5]),.1)) (round((r(table)[6,5]),.1))
        
        di "Compare to tabular data:"
        table iso5, c(count `var' mean `var' sem `var') 

    di "----- PERCENT DIFFERENCE MODEL -----"
    * Rate difference models
        nbreg `var' levels* homeorder i.rucc, irr vce(r)   
           
   * Plot graph
       frame change results
       la var level "Social Distancing: Lowest (1) to Highest (5)"
       line avg level if inlist(strat,"`var'"), title("`var'")

    frame change default
    
    }
end

// Set up scaled Poisson model

program define modelpoisson, rclass
    version 16
    syntax varlist(numeric)
    
    frame change default
    foreach var of local varlist {

    di "----- RURALITY-ADJUSTED POISSON MODEL -----"
       glm `var' levels* homeorder i.rucc, family(poisson) link(log) scale(x2) eform nocons 

    * Store results
        frame post results ("`var'") (1) (round((r(table)[1,1]),.1)) (round((r(table)[5,1]),.1)) (round((r(table)[6,1]),.1))
        frame post results ("`var'") (2) (round((r(table)[1,2]),.1)) (round((r(table)[5,2]),.1)) (round((r(table)[6,2]),.1))
        frame post results ("`var'") (3) (round((r(table)[1,3]),.1)) (round((r(table)[5,3]),.1)) (round((r(table)[6,3]),.1))
        frame post results ("`var'") (4) (round((r(table)[1,4]),.1)) (round((r(table)[5,4]),.1)) (round((r(table)[6,4]),.1))
        frame post results ("`var'") (5) (round((r(table)[1,5]),.1)) (round((r(table)[5,5]),.1)) (round((r(table)[6,5]),.1))
        
        di "Compare to tabular data:"
        table iso5, c(count `var' mean `var' sem `var') 

    di "----- PERCENT DIFFERENCE MODEL -----"
    * Rate difference models
       glm `var' levels* homeorder i.rucc, family(poisson) link(log) scale(x2) eform 
           
   * Plot graph
       frame change results
       la var level "Social Distancing: Lowest (1) to Highest (5)"
       line avg level if inlist(strat,"`var'"), title("`var'")

    frame change default
    
    }
end



---

# Descriptive Results

In [24]:
// Basic distributions of counties and traces
* iso5 is the main outcome variable representing quintiles of mobility change

tab rucc, m
tab iso5, m
table iso5, c(sum last3_sample)
su last3_sample
di "Total 3-day mobile traces: " r(sum)

* US average change in mobility
qui: su last3_index
    return list

* Mean mobility change by quintile
table iso5, c(count last3_index mean last3_index sem last3_index sum samples median last3_sample) 

* Mean mobility change by status of homeorder
table homeorder, c(mean last3_index sem last3_index)

* Rurality/urbanicity by quintile
table iso5, c(median rucc)



       rucc |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |        345       16.27       16.27
          2 |        309       14.58       30.85
          3 |        273       12.88       43.73
          4 |        171        8.07       51.79
          5 |         76        3.58       55.38
          6 |        436       20.57       75.94
          7 |        322       15.19       91.13
          8 |         94        4.43       95.57
          9 |         94        4.43      100.00
------------+-----------------------------------
      Total |      2,120      100.00


Distancing: |
 Lowest (1) |
 to Highest |
        (5) |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |        421       19.86       19.86
          2 |        422       19.91       39.76
          3 |        423       19.95       59.72
          4 |        429       20.24       79.95
          5 |        425       20.0

r(111);
r(111);


---
# Healthcare

To gauge overall baseline healthcare access and utilization, we examined primary care providers per 100,000 population and percent uninsured under age 65 (e.g., Medicare eligibility). As a marker for a closely related preventive health behavior, we examined whether earlier influenza vaccination rates were associated with how much the county was likely to slow down in the current coronavirus outbreak. This was quantified as the percent of annual Medicare enrollees having an annual influenza vaccination.

## Primary care providers

We wanted to see if places with more social distancing had better healthcare resources. So we looked at primary care providers per 100,000 population. 


In [ ]:
modelrun pcp_rate

### Interpretation
<br><br>

Relative effect measures <br>
`    ^
     levels1 |    49.4269   2.469613    78.06   0.000       44.816    54.51219
     levels2 |   50.90777   2.527614    79.15   0.000     46.18715    56.11086
     levels3 |   56.66436   2.832643    80.76   0.000      51.3758    62.49733
     levels4 |   62.54773   2.997007    86.32   0.000     56.94109    68.70642
     levels5 |   72.75614   3.547062    87.94   0.000     66.12584    80.05124
`
<br><br>
Percent difference<br> 
`levels1 |   .6793503    .027583    -9.52   0.000     .6273836    .7356213`

In [ ]:
invert .6793503 .6273836 .7356213


The lowest social distancing counties had 49.4 primary care providers per 100,000, whereas the most social distancing counties had 72.7 per 100,000 after adjusting for rurality and social distancing orders, a 47% (95% CI: 36%, 59%) difference. 

---

## Percent uninsured
Percent of without health insurance below Medicare elgibility (age 65).

In [ ]:
// Comparing percent uninsured to social distancing
modelpoisson uninsured_p 

### Interpretation
<br><br>

Relative effect measures <br>
`    ^
     levels1 |   11.31834   .4589692    59.84   0.000      10.4536    12.25462
     levels2 |   10.65563    .429903    58.65   0.000      9.84549    11.53244
     levels3 |   9.304829   .3793602    54.71   0.000     8.590228    10.07888
     levels4 |   8.544709   .3466857    52.88   0.000     7.891533    9.251949
     levels5 |   7.267551   .2956462    48.76   0.000     6.710594    7.870734
`
<br><br>
Percent difference<br> 
`levels1 |   1.557381    .047272    14.59   0.000     1.467431    1.652844`

Counties with lower social distancing also had a higher proportion of people without health insurance. The lowest social distancing counties had 11.3% uninsured adults, whereas the most social distancing counties had only 7.27% uninsured after adjusting for rurality and social distancing orders, a 56% (95% CI: 47%, 65%) difference. 

---

## Flu Vaccination

We had a hypothesis that counties that were more involved in preventative behaviors would be more likely to self-isolate more thoroughly. To test this, we examined whether earlier flu vaccination rates impacted how much the county was likely to slow down in the current coronavirus outbreak. This is quantified as the percent of annual Medicare enrollees having an annual flu vaccination, as reported by the Robert Wood Johnson Foundation. Since the flu vaccine is free to all Medicare beneficiaries, and this is the elderly age group with the most influenza mortality, this is a convenient metric to test *a priori* how conscientious the population was, on average.

In [ ]:
// Basic descriptive on background influenza vaccine
frame change default
summ fluvaccine, d
hist fluvaccine, bin(10)

In [ ]:
// Comparing background flu vaccination with current social distancing
modelpoisson fluvaccine

### Interpretation
<br><br>

Relative effect measures <br>
`    ^
     levels1 |   47.85711    .885275   209.11   0.000     46.15308    49.62405
     levels2 |   47.61874   .8674556   212.07   0.000     45.94855    49.34964
     levels3 |   48.93908   .8869371   214.67   0.000     47.23123    50.70869
     levels4 |   49.81001    .887262   219.40   0.000     48.10102    51.57973
     levels5 |   50.76095    .885802   225.04   0.000     49.05417    52.52713
`
<br><br>
Percent difference<br> 
`levels1 |   .9427937   .0129735    -4.28   0.000     .9177059    .9685673`


The lowest social distancing counties had 47.8% flu vaccine coverage among Medicare beneficiaries, whereas the most social distancing counties had 50.8% after adjusting for rurality and social distancing orders, a 6.1% (95% CI: 3.2%, 9.0%) difference. 

In [ ]:
invert .9427937 .9177059 .9685673

--- 
# Economic

There is a trend emerging. So, since the places with more social distancing seem to have more health resources, perhaps there are trends in financial means? We explored two baseline economic metrics in relation to social distancing, one representing the overall wealth of the community and one proxy for poverty: 80th percentile of annual household income in dollars and the percent of school-age children eligible for subsidized or free lunches.

## Household income

In [ ]:
// Comparing 80th percentile income to social distancing
modelrun income80

### Interpretation
<br><br>

Relative effect measures <br>
`    ^
     levels1 |   120578.2    2174.73   648.71   0.000     116390.3    124916.8
     levels2 |   121964.6   2103.959   678.90   0.000     117909.8    126158.7
     levels3 |   127609.2   2292.704   654.37   0.000     123193.8    132182.9
     levels4 |     132518   2338.222   668.45   0.000     128013.5      137181
     levels5 |   140685.7   2682.181   621.78   0.000     135525.8    146042.2
`
<br><br>
Percent difference<br> 
`levels1 |   .8570749   .0113487   -11.65   0.000     .8351179    .8796091`


The lowest social distancing counties the 80th percentile of annual houshold income was `$120,578`, whereas in the most social distancing counties it was `$140,685`, after adjusting for rurality and social distancing orders, a 17% (95% CI: 14%, 20%) difference. 


In [ ]:
invert .8570749 .8351179 .8796091

---

## Subsidized lunches

In [ ]:
modelrun schoollunch

### Interpretation
<br><br>

Relative effect measures <br>
`    ^
     levels1 |   41.84078   1.429248   109.31   0.000     39.13122    44.73796
     levels2 |   41.01835   1.338739   113.80   0.000     38.47663    43.72797
     levels3 |   37.65352   1.314916   103.90   0.000     35.16255    40.32095
     levels4 |   35.03891   1.189645   104.75   0.000     32.78313     37.4499
     levels5 |   33.73666   1.146257   103.56   0.000     31.56321    36.05978
`
<br><br>
Percent difference<br> 
`levels1 |   1.240217   .0277847     9.61   0.000     1.186938    1.295887`


In the lowest social distancing counties, 42% of schoolage children were eligible for free or reduced price lunches. By comparison, in the the most social distancing counties 34% were eligible, after adjusting for rurality and social distancing orders, a 24% (95% CI: 19%, 29%) difference. 


---
# Structural
Three lifestyle metrics were selected to provide a diverse snapshot of baseline structural factors that could influence defiance of prolonged stay-at-home orders. The percent of people experiencing food insecurity was derived from Map the Meal Gap project, based on responses from the Current Population Survey and a cost-of-food index. Access to exercise opportunities was the percent of population with adequate access to locations for physical activity. The percent of households with overcrowding was based on the Comprehensive Housing Affordability Strategy measurements.

## Food insecurity

In [ ]:
modelpoisson foodinsec

### Interpretation
<br><br>

Relative effect measures <br>
`    ^
     levels1 |   13.16074    .371045    91.41   0.000     12.45323    13.90844
     levels2 |    12.5652   .3522368    90.28   0.000     11.89345    13.27489
     levels3 |   11.63859   .3279836    87.09   0.000     11.01318    12.29951
     levels4 |    10.9946   .3073966    85.75   0.000     10.40833     11.6139
     levels5 |   10.41177   .2879299    84.72   0.000     9.862455    10.99167
`
<br><br>
Percent difference<br> 
`levels1 |   1.264025    .025903    11.43   0.000     1.214263    1.315828`


The lowest social distancing counties had greater food insecurity, among 13.2% of residents. The most social distancing counties had 10.4%, after adjusting for rurality and social distancing orders, a 26.4% (95% CI: 21.4%, 31.5%) difference. 

---

## Exercise opportunities

In [ ]:
modelrun exercise

### Interpretation
<br><br>

Relative effect measures <br>
`    ^
     levels1 |   69.32704   2.008161   146.34   0.000     65.50076    73.37684
     levels2 |   73.66608    1.99158   159.03   0.000     69.86427    77.67477
     levels3 |   78.57573   2.050318   167.25   0.000     74.65821    82.69881
     levels4 |   82.82771   2.039621   179.36   0.000     78.92507    86.92334
     levels5 |   92.01346   2.177853   191.05   0.000     87.84245    96.38254
`
<br><br>
Percent difference<br> 
` levels1 |   .7534445    .017626   -12.10   0.000     .7196781    .7887951`


In the lowest social distancing counties, 69% of residents had access to physical spaces for exercise, whereas in the most social distancing counties 92% had access, after adjusting for rurality and social distancing orders, a 33% (95% CI: 27%, 39%) difference. 



In [ ]:
invert .7534445 .7196781 .7887951

---

## Overcrowding

In [ ]:
modelpoisson overcrowding

### Interpretation
<br><br>

Relative effect measures <br>
`    ^
     levels1 |   2.386115    .182761    11.35   0.000       2.0535    2.772604
     levels2 |   2.272371   .1727998    10.79   0.000     1.957721    2.637594
     levels3 |   2.148765   .1640225    10.02   0.000      1.85018    2.495537
     levels4 |   2.101588   .1584565     9.85   0.000     1.812877    2.436278
     levels5 |   2.055545   .1525507     9.71   0.000     1.777279    2.377378
`
<br><br>
Percent difference<br> 
`levels1 |   1.160819   .0639501     2.71   0.007     1.042008    1.293176`


The lowest social distancing counties had 16% (95% CI: 4.2%, 29%) less overcrowding, after adjusting for rurality and social distancing orders. 

---

# Exploratory analyses

In [ ]:
frame change default
foreach var of varlist drivealone_p {
    table iso5, c(count `var' mean `var' sem `var')
        frame put `var' iso5, into(`var')
            frame change `var'
                collapse (mean) `var', by(iso5)
                    la var `var' "% of Drivers"
                        line `var' iso5, note("Commuting Alone by Vehicle")  
                            frame change default
                                frame drop `var'
}

---


In [ ]:
frame change default
foreach var of varlist rucc {
    table iso5, c(count `var' mean `var' sem `var')
        frame put `var' iso5, into(`var')
            frame change `var'
                collapse (median) `var', by(iso5)
                    la var `var' "Meidan RUCC"
                        line `var' iso5, note("Urban-Rural")   
                            frame change default
                                frame drop `var'
}

In [ ]:
frame change default
foreach var of varlist longcommute_p {
    table iso5, c(count `var' mean `var' sem `var')
        frame put `var' iso5, into(`var')
            frame change `var'
                collapse (mean) `var', by(iso5)
                    la var `var' "% of Solo Commuters Driving 30+ mins"
                        line `var' iso5, note("Long Solo Commute")     
                            frame change default
                                frame drop `var'
}

---

# External validation with Google data

Correlations between March 1 to April 11 in Google and DL data by county-day

In [ ]:
// Import Descartes Labs data
clear
import delimited "https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-mobility-daterow.csv", encoding(ISO-8859-9) stringcols(6) 

    * Drop state aggregates
        drop if admin2==""

    * Format date
        gen date2=date(date,"YMD")
            format date2 %td
                drop date
                    rename date2 date

    * Note data start and end dates for graphs
        su date 
            local latest: disp %td r(max)
                di "`latest'"
            local earliest: disp %td r(min)
                di "`earliest'"

    * Rename variables for consistency
        rename admin1 state
        rename admin2 county

save dl_x_valid, replace

// Process Google app check-in data
clear
import delimited "/Users/nabarun/Documents/GitHub/covid/fips-google-mobility-daily-as-of-04-20-20.csv", stringcols(1) numericcols(5 6 8)  

    * Format date
        gen date=date(report_date, "YMD")
            format date %td
                order date, first
                    drop report_date

    * Note data start and end dates for graphs
        su date 
            local latest: disp %td r(max)
                di "Latest: " "`latest'"
            local earliest: disp %td r(min)
                di "Earliest: " "`earliest'"

    save google_x_valid, replace
    

// Merge by date and county, retain
    merge 1:1 date fips using dl_x_valid

    tab _merge
        keep if _merge==3
            drop _merge country_code admin_level

    * Variable cleanup
    destring retail_and_recreation_percent_ch grocery_and_pharmacy_percent_cha workplaces_percent_change_from_b, replace force
    rename retail_and_recreation_percent_ch retailrec
    rename grocery_and_pharmacy_percent_cha grocery
    rename workplaces_percent_change_from_b work
    rename residential_percent_change_from_ home
    rename parks_percent_change_from_baseli parks
        la var parks "Parks"
    rename transit_stations_percent_change_ transit
                la var transit "Transit"
    
    gen m50i = m50_index-100
            la var m50i "Re-centered at zero"

    * Correlation matrix on complete case data (n=20,891)
    correlate m50i retailrec grocery work home parks transit
        matrix C = r(C)
     
    * Pairwise correlations
    correlate m50i retailrec
    correlate m50i grocery
    correlate m50i work
    correlate m50i home
    correlate m50i parks
    correlate m50i transit

---

*fin.*